In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))


import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

image = tf.zeros([1, 200, 200, 100])

lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2

# warm up
lstm_cell(input, state); lstm_fn(input, state)



class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

@tf.function
def f(x):
  if x > 0:
    # Try setting a breakpoint here!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist()

model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0

  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy


step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())



Step 10 : loss 1.78687608 ; accuracy 0.409
Step 20 : loss 1.23370647 ; accuracy 0.5455
Step 30 : loss 0.751195729 ; accuracy 0.62
Step 40 : loss 0.744434655 ; accuracy 0.67525
Step 50 : loss 0.39180249 ; accuracy 0.7092
Step 60 : loss 0.398978919 ; accuracy 0.736833334
Step 70 : loss 0.384810954 ; accuracy 0.759571433
Step 80 : loss 0.283165574 ; accuracy 0.776875
Step 90 : loss 0.271611214 ; accuracy 0.790333331
Step 100 : loss 0.316256613 ; accuracy 0.8008
Step 110 : loss 0.274215162 ; accuracy 0.810090899
Step 120 : loss 0.203440234 ; accuracy 0.818333328
Step 130 : loss 0.233346179 ; accuracy 0.825692296
Step 140 : loss 0.206543267 ; accuracy 0.833214283
Step 150 : loss 0.33658573 ; accuracy 0.839
Step 160 : loss 0.347389758 ; accuracy 0.843937516
Step 170 : loss 0.26370573 ; accuracy 0.848470569
Step 180 : loss 0.351972729 ; accuracy 0.852666676
Step 190 : loss 0.232818142 ; accuracy 0.855789483
Step 200 : loss 0.302266836 ; accuracy 0.8589
Final step tf.Tensor(200, shape=(), dtyp